## Wikipedia Scrapping

In [1]:
import pandas as pd

In [10]:
#read the wiki page
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [22]:
#drop rows where Borough is not assigned
toronto_df = toronto_df.drop(toronto_df[toronto_df['Borough']=='Not assigned'].index)

In [40]:
toronto_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [39]:
toronto_df.groupby(['Postal code','Borough'])['Neighborhood']

2                                              Parkwoods
3                                       Victoria Village
4                             Regent Park / Harbourfront
5                      Lawrence Manor / Lawrence Heights
6           Queen's Park / Ontario Provincial Government
8                                       Islington Avenue
9                                        Malvern / Rouge
11                                             Don Mills
12                      Parkview Hill / Woodbine Gardens
13                              Garden District, Ryerson
14                                             Glencairn
17     West Deane Park / Princess Gardens / Martin Gr...
18              Rouge Hill / Port Union / Highland Creek
20                                             Don Mills
21                                      Woodbine Heights
22                                        St. James Town
23                                    Humewood-Cedarvale
26     Eringate / Bloordale Gar